In [ ]:
# import libraries
import boto3
import os
from io import StringIO

In [ ]:
import pandas as pd

In [ ]:
%run lib/__wesley_init_verbose__.py

In [ ]:
# !pip install s3fs

In [ ]:
# os.environ['AWS_PROFILE'] = "profile"
# os.environ['AWS_DEFAULT_REGION'] = "us-east-1"

# create client object
s3 = boto3.client('s3')

In [ ]:
# list all s3 buckets in json format
response = s3.list_buckets()
print(response)

In [ ]:
# check S3 bucket
# print s3 bucket to make sure linked

def status_df(response):
    return pd.DataFrame({
    'ts': [(x['CreationDate']) for x in response['Buckets']],
    'bucket name': [x['Name'] for x in response['Buckets']]  
})

status_df(response)

In [ ]:
# create resource object
s3_resource = boto3.resource('s3')

In [ ]:
# access file from s3 include bucket name and key
obj = s3_resource.Object(bucket_name='sandy-hook-data', key='sandy_one_year copy.csv')

# use stringIO to read object
s3_data = StringIO(obj.get()['Body'].read().decode('utf-8'))

In [ ]:
# pull data into pandast dataframe and read csv
data = pd.read_csv(s3_data)
print(data.head())

In [ ]:
# ben's code
# Import the books.csv file as a DataFrame
# why show vertical instead of horizontal?
df = pd.read_csv('../sandy_one_year copy.csv')
df.sample(3).T

In [ ]:
print('From the "../sandy_one_year copy.csv" file.\n')
print('CSV File Shape:\n',df.shape)
print('\n\nCSV Column Titles:\n',df.columns)
# print('\n\nHashtag Value Counts:\n', df.hashtags.value_counts())

#--------------------------------------------
# Make hashtag column into list of values

all_tags = []
for b in df['hashtags']:
    c = str(b).split(", ")
    c = str(c).replace('[', '')
    c = str(c).replace(']', '')
    c = str(c).replace("'", '')
    c = str(c).replace('"', '')
    c = str(c).split(', ')
    for element in c:
        all_tags.append(element)
            
# ----------------------------------------------
# Find the top 10 hashtags

from collections import Counter

hashtag_count = dict(Counter(all_tags))

print('\n\nTop 10 Most Frequent Hashtags: \n', sorted(hashtag_count, key=hashtag_count.get, reverse=True)[:10])
#  or 
print('\n\nCount of Top 10 Most Frequent Hashtags: \n',dict(Counter(hashtag_count).most_common(10)))

# --- username frequency ------------------------

usernames = []
for b in df['username']:
    usernames.append(b)

username_count = dict(Counter(usernames))

print('\n\nTop 10 Most Frequent Usernames: \n', sorted(username_count, key=username_count.get, reverse=True)[:10])
#  or 
print('\n\nCount of Top 10 Most Frequent Usernames: \n',dict(Counter(username_count).most_common(10)), '\n\n',)

display(df.info())

In [ ]:
# 'EDT', 'Eastern Standard Time' and 'Eastern Daylight Time'

'''preparing a csv to use for further analysis'''

# ---Import libraries---
%run lib/__wesley_init__.py
import pandas as pd
from collections import Counter
from datetime import *
from tqdm import tqdm
from decimal import Decimal

print('Libraries loaded') #Progress check 


# ---Functions---

def hashtags_used_count(username):
    from collections import Counter
    username_sorted_df = df[df['username'] == username]
    all_tags = []
    for b in username_sorted_df['hashtags']:
        c = str(b).split(", ")
        c = str(c).replace('[', '')
        c = str(c).replace(']', '')
        c = str(c).replace("'", '')
        c = str(c).replace('"', '')
        c = str(c).split(', ')
        for element in c:
            all_tags.append(element)
    
    hashtag_count = dict(Counter(all_tags))
    return hashtag_count

def min_max_date(username):
    from collections import Counter
    username_sorted_df = df[df['username'] == username]
    username_sorted_df = username_sorted_df.reset_index(drop=True)
    min_date = min(username_sorted_df['date'])
    max_date = max(username_sorted_df['date'])
    return min_date, max_date

def count_days(min_date, max_date):
    date_format = "%Y-%m-%d"
    min_date = datetime.strptime(min_date, date_format)
    max_date = datetime.strptime(max_date, date_format)
    delta = max_date - min_date
    return delta.days

def extract_mentions(df):
    mentions = []
    for tweet in df['tweet']:
        tokened_tweet = tokenize_by_words2(tweet)
        for word in tokened_tweet:
            if '@' in word:
                mentions.append(word)
    mention_count = Counter(mentions)
    return mention_count

# change to eastern/standard/daylight savings
#  df.loc[iterator, 'created_at'] = df.loc[iterator, 'created_at'].replace(' PDT', '')
#             df.loc[iterator, 'created_at'] = df.loc[iterator, 'created_at'].replace(' Pacific Standard Time', '')
#             df.loc[iterator, 'created_at'] = df.loc[iterator, 'created_at'].replace(' Pacific Daylight Time', '')
def prep_created_at_column(df, csv_name):
    iterator = 0
    counter = 0
    fmt = '%Y-%m-%d %H:%M:%S'
    for _ in tqdm(df.created_at):
        if counter < 10000: #This is an autosave feature for really large files
            df.loc[iterator, 'created_at'] = df.loc[iterator, 'created_at'].replace(' EST', '')
            df.loc[iterator, 'created_at'] = df.loc[iterator, 'created_at'].replace(' EDT', '')
            df.loc[iterator, 'created_at'] = df.loc[iterator, 'created_at'].replace(' Eastern Standard Time', '')
            df.loc[iterator, 'created_at'] = df.loc[iterator, 'created_at'].replace(' Eastern Daylight Time', '')
            iterator += 1
            counter += 1
        else:
            df.to_csv(csv_name+'.csv')
            counter = 0
    df['created_at'] = pd.to_datetime(df['created_at'],infer_datetime_format=True )

def avg_minutes_between_tweets(df):
    num_list = []
    iterator = 0
    for _ in range(len(df)-1):
        diff = (df.loc[iterator, 'created_at']) - (df.loc[iterator+1, 'created_at'])
        diff_minutes = (diff.days * 24 * 60) + (diff.seconds/60)
        if diff_minutes < 0:
            diff_minutes = 0
        num_list.append(diff_minutes)
        iterator += 1
    try: avg = sum(num_list) / len(num_list)
    except ZeroDivisionError: avg = 0
    return float(avg), min(num_list), max(num_list)

print('Functions loaded') #Progress Check

# ---Establish dataframes---

file = input('Which file name would you like me to read (minus the suffix)? ')

df = pd.read_csv(file+'.csv')
print('Main DataFrame loaded') #Progress Check
print('Changing "created_at" column to datetime')
prep_created_at_column(df, file+'2')
print('Created_at column changed') #Progress Check
df.to_csv(file+'2.csv')
refined_df = pd.DataFrame()

print('DataFrames Loaded') #Progress Check


#---Processing script---

# --- username frequency ---

usernames = []
for b in df['username']:
    usernames.append(b)

username_count = dict(Counter(usernames))

print('username count completed')

# --- processing ---
iterator = 0
for handle in tqdm(sorted(username_count, key=username_count.get, reverse=True)):
    print('Processing:', handle) #Progress Check
    temp_df = df[df['username'] == handle] #Establishes a temp sorted df of each username encountered
    print(handle, 'DataFrame established') #Progress Check
    temp_df = temp_df.reset_index(drop=True)
    refined_df.loc[iterator, 'username'] = handle
    refined_df.loc[iterator, 'total_tweets'] = int(len(temp_df))
    refined_df.loc[iterator, 'total_retweets'] = [dict(Counter(list(temp_df['retweet'])))]
    refined_df.loc[iterator, 'hashtag_frequency'] = [dict(Counter(hashtags_used_count(handle)).most_common())]
    try: min_date, max_date = min_max_date(handle)
    except ValueError: continue
    refined_df.loc[iterator, 'min_date'] = (min_date)
    refined_df.loc[iterator, 'max_date'] = (max_date)
    refined_df.loc[iterator, 'date_range_day_count'] = count_days(min_date, max_date)
    mentions = [extract_mentions(temp_df)]
    refined_df.loc[iterator, 'mentions'] = mentions
    try: avg, min_between, max_between = avg_minutes_between_tweets(temp_df)
    except ValueError: continue
    refined_df.loc[iterator, 'avg_min_between_tweets'] = float(round(Decimal(str(avg)), 2))
    refined_df.loc[iterator, 'min_min_between_tweets'] = float(round(Decimal(str(min_between)), 2))
    refined_df.loc[iterator, 'max_min_between_tweets'] = float(round(Decimal(str(max_between)), 2))
    
    iterator += 1
    
refined_df.head()